Renaming is done 

In [1]:
from elasticsearch import Elasticsearch
from elasticsearch.client import IndicesClient
from elasticsearch.client import CatClient
from elasticsearch.client import TasksClient

In [2]:
elastic_db = Elasticsearch(['localhost:9200'])
indices_client = IndicesClient(elastic_db)
cat_client = CatClient(elastic_db)
task_client = TasksClient(elastic_db)

In [3]:
# To view the indices on Elastisearch:
print(cat_client.indices(v=True))

health status index   uuid                   pri rep docs.count docs.deleted store.size pri.store.size
yellow open   .kibana _RRwYu7rSd2H7CuBLtZ4dw   1   1          2            1     12.2kb         12.2kb
yellow open   src_idx CjnMn89WQEqtaLZwixTLlw   5   1        256            0    124.4kb        124.4kb



From Elasticsearch documentation:  
https://www.elastic.co/guide/en/elasticsearch/reference/current/docs-reindex.html  
Reindex does not attempt to set up the destination index.  
It does not copy the settings of the source index.  
You should set up the destination index prior to running a \_reindex action, including setting up mappings, shard counts, replicas, etc.  
  
src_idx has to be renamed to dest_idx

In [4]:
# Create the dest_idx with required settings and mappings
idx_structure = {
    "settings":{
    },
    "mappings": {
        "doc": {
            "properties": {
                "user_name": {
                    "type": "text",
                    "fields": {
                        "exact": { "type": "keyword" }
                    }
                },
                "user_id" : { "type" : "keyword" },
                "timestamp" : { "type" : "date" },
                "user_status" : { "type" : "byte" }
            }
        }
    }
}

response = indices_client.create(index='dest_idx',body=idx_structure)
print(response)

{'shards_acknowledged': True, 'index': 'dest_idx', 'acknowledged': True}


In [5]:
# Perform reindex operation
req_body = {
    "source": {
        "index": "src_idx"
    },
    "dest": {
        "index": "dest_idx"
    }
}

response = elastic_db.reindex(body=req_body, refresh=True,wait_for_completion=False)
print(response)

{'task': 'CWaF_uEVS1GM12CFVmVzxQ:6767'}


In [7]:
# Check the status of the reindex operation
response = task_client.get(task_id='CWaF_uEVS1GM12CFVmVzxQ:6767') # Use the task id from above cell's response
print(response['task']['description'])

reindex from [src_idx] to [dest_idx]


In [8]:
# To view the indices on Elastisearch:
print(cat_client.indices(v=True))

health status index    uuid                   pri rep docs.count docs.deleted store.size pri.store.size
yellow open   dest_idx hjOdWrMdRNOnscMWhHmRtw   5   1        256            0    133.6kb        133.6kb
yellow open   .kibana  _RRwYu7rSd2H7CuBLtZ4dw   1   1          2            1     12.2kb         12.2kb
yellow open   src_idx  CjnMn89WQEqtaLZwixTLlw   5   1        256            0    124.4kb        124.4kb
green  open   .tasks   nfz1s74SRxaUu388nL3MYA   1   0          1            0      6.1kb          6.1kb



In [9]:
# Delete the source index
response = indices_client.delete('src_idx')
print(response)

{'acknowledged': True}


In [10]:
# Delete the tasks
response = indices_client.delete('.tasks')
print(response)

{'acknowledged': True}


In [11]:
# To view the indices on Elastisearch:
print(cat_client.indices(v=True))

health status index    uuid                   pri rep docs.count docs.deleted store.size pri.store.size
yellow open   dest_idx hjOdWrMdRNOnscMWhHmRtw   5   1        256            0    133.6kb        133.6kb
yellow open   .kibana  _RRwYu7rSd2H7CuBLtZ4dw   1   1          2            1     12.2kb         12.2kb



Thus we have renamed src_idx as dest_idx